#### Não precisa olhar esse código do Nixxla. Só fiz ele pra mostrar pra vocês os códigos do PredictionManager rodando

In [5]:
# from mlforecast import MLForecast
import pandas as pd

In [52]:
path = r'C:\Users\micha\Documents\GitHub\Projeto-PET-Sauter\data\M5_full.parquet'

dados = pd.read_parquet(path)

In [53]:
dados['date'] = pd.to_datetime(dados['date'])

In [54]:
dados['year_month'] = dados['date'].dt.to_period('M')

In [55]:
dados['year_month'] = dados['year_month'].dt.to_timestamp()

In [56]:
df_general = dados.groupby('year_month')['value'].sum().reset_index().set_index('year_month')

In [57]:
df_hobbies = dados.groupby(['cat_id', 'year_month'])['value'].sum().reset_index().set_index('year_month')
df_hobbies = df_hobbies[df_hobbies['cat_id'] == 'HOBBIES']

C:\Users\micha\AppData\Local\Temp\ipykernel_7064\13977740.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_hobbies = dados.groupby(['cat_id', 'year_month'])['value'].sum().reset_index().set_index('year_month')


In [58]:
df_nixtla_general = pd.DataFrame({'ds': df_general.index, 'y': df_general['value'].values, 'unique_id' : 'general'})
df_nixtla_hobbies = pd.DataFrame({'ds': df_hobbies.index, 'y': df_hobbies['value'].values, 'unique_id': 'hobbies'})

df_nixtla = pd.concat([df_nixtla_general, df_nixtla_hobbies], ignore_index=True)

In [59]:
test_size = (int)(df_nixtla.shape[0]*0.2)

df_test = df_nixtla.tail(test_size)
df_train = df_nixtla.drop(df_test.index)    

In [60]:
from sklearn.base import BaseEstimator

class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

In [61]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [62]:
selected_models = [
    DecisionTreeRegressor(),
    LinearRegression(),
    Naive()
]

In [63]:
ml_forecast = MLForecast(
    models = selected_models,
    freq = 'M',
    lags=[1,2,3,4,5,6]
)

In [64]:
ml_forecast.fit(df_nixtla)

MLForecast(models=[DecisionTreeRegressor, LinearRegression, Naive], freq=M, lag_features=['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6'], date_features=[], num_threads=1)

##### Aqui é o que precisa ser salvo

In [65]:
predict_full = ml_forecast.predict(6)

C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\Python\Python311\site-packages\utilsforecast\processing.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
C:\Users\micha\AppData\Roaming\P

#### Vocês precisam usar isso aqui pra fazer o dashboard

##### Essa classe serve pra gerenciar as predições no nosso Banco de Dados de predição. Basicamente, serve como uma API pra salvar e consultar as predições feitas pelo Nixxla.

In [1]:
from PredictionsManager import PredictionsManager

##### Esse path pro db eu criei pra testar a classe, então criem onde ficar melhor pra rodar o dashboard

In [2]:
db_path = 'predictions.db'

In [3]:
pred_manager = PredictionsManager(db_path=db_path)

Banco de dados conectado: predictions.db


##### Vejam que o meu DataFrame 'pred_full' tem as previsões de todos os modelos que eu selecionei pra cada uma dos unique_id

In [69]:
predict_full.head(5)

,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
0,general,2016-05-31,1054534.0,1.075692e+06,994982.0
1,general,2016-06-30,1057012.0,1.040079e+06,994982.0
2,general,2016-07-31,1013452.0,1.040280e+06,994982.0
3,general,2016-08-31,1123706.0,1.179614e+06,994982.0
4,general,2016-09-30,1104287.0,1.099268e+06,994982.0


In [70]:
predict_full.tail(5)

,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
7,hobbies,2016-06-30,94810.0,102129.148438,91658.0
8,hobbies,2016-07-31,94810.0,101637.734375,91658.0
9,hobbies,2016-08-31,99818.0,118622.085938,91658.0
10,hobbies,2016-09-30,98066.0,110839.406250,91658.0
11,hobbies,2016-10-31,104388.0,116269.210938,91658.0


##### Pra salvar essas previsões na base de dados basta passar esse o df_nixtla q a gnt usou pra fazer as previsões e o predict, que o manager vai separar cada previsão em uma tabela com o nome correspondente ao 'unique_id' dela em 'predict_full'. Exemplo: pra o dataframe 'general', o manager vai criar uma tabela 'general' com o df original e uma tabela 'prev_general' com a previsao.

In [71]:
pred_manager.save_predictions(df_nixtla, predict_full)

Todas as previsões foram salvas em tabelas chamadas: ['general' 'hobbies']


##### Pra pegar as previsões, é só usar o método abaixo e pasasr o 'unique_id' correspondente a previsão que você quer e ele vai retornar o dataframe original e o dataframe da previsao. Seguindo o exemplo acima, se vc quiser pegar a previsao pra general, vc vai botar a string 'general' e receber o dataframe original e a previsao dele numa tupla nessa ordem.

In [72]:
original_series, prev_hobbies = pred_manager.get_prediction('general')
original_series.head()

Previsões recuperadas com sucesso para: general


,ds,y,unique_id
0,2011-01-01,88163,general
1,2011-02-01,726375,general
2,2011-03-01,763567,general
3,2011-04-01,737713,general
4,2011-05-01,719562,general


In [73]:
prev_hobbies.head()

,unique_id,ds,DecisionTreeRegressor,LinearRegression,Naive
0,general,2016-05-31,1054534.0,1.075692e+06,994982.0
1,general,2016-06-30,1057012.0,1.040079e+06,994982.0
2,general,2016-07-31,1013452.0,1.040280e+06,994982.0
3,general,2016-08-31,1123706.0,1.179614e+06,994982.0
4,general,2016-09-30,1104287.0,1.099268e+06,994982.0


##### Depois de usar essa classe pra salvar/pegar algum dado no BD, lembrem de fechar a conexão.

In [11]:
pred_manager.close_connection()

Conexão fechada


##### Adicionando previsões

In [34]:
path_pred = "../../data/predictions.parquet"
path_nix = "../../data/all_M5_series.parquet"

df_nix = pd.read_parquet(path_nix)
df_pred = pd.read_parquet(path_pred)

In [35]:
df_nix['ds'] = df_nix['ds'].dt.to_timestamp()

In [36]:
df_nix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982955 entries, 0 to 1982954
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 45.4+ MB


In [37]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ds                462 non-null    datetime64[ns]
 1   AutoARIMA         462 non-null    float32       
 2   ETS               462 non-null    float32       
 3   Theta             462 non-null    float32       
 4   unique_id         462 non-null    object        
 5   LGBMRegressor     462 non-null    float32       
 6   XGBRegressor      462 non-null    float32       
 7   LinearRegression  462 non-null    float32       
dtypes: datetime64[ns](1), float32(6), object(1)
memory usage: 18.2+ KB


In [38]:
db_path = 'predictions.db'

In [42]:
pred_manager = PredictionsManager(db_path=db_path)

Banco de dados conectado: predictions.db


In [40]:
pred_manager.save_predictions(df_nix, df_pred)

Todas as previsões foram salvas em tabelas chamadas: ['RTM' 'CA' 'TX' 'WI' 'CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3'
 'WI_1' 'WI_2' 'WI_3' 'FOODS' 'HOBBIES' 'HOUSEHOLD'
 'FOODS_3_090_CA_3_evaluation' 'FOODS_3_586_TX_2_evaluation'
 'FOODS_3_586_TX_3_evaluation' 'FOODS_3_586_CA_3_evaluation'
 'FOODS_3_090_CA_1_evaluation' 'FOODS_3_090_WI_3_evaluation'
 'FOODS_3_090_TX_2_evaluation' 'FOODS_3_090_TX_3_evaluation'
 'FOODS_3_252_TX_2_evaluation' 'FOODS_3_586_TX_1_evaluation'
 'HOBBIES_1_234_CA_3_evaluation' 'HOBBIES_1_348_CA_1_evaluation'
 'HOBBIES_1_371_CA_1_evaluation' 'HOBBIES_1_254_CA_3_evaluation'
 'HOBBIES_1_371_CA_3_evaluation' 'HOBBIES_1_348_CA_3_evaluation'
 'HOBBIES_1_268_CA_1_evaluation' 'HOBBIES_1_341_CA_3_evaluation'
 'HOBBIES_1_015_CA_3_evaluation' 'HOBBIES_1_268_CA_3_evaluation'
 'HOUSEHOLD_1_118_CA_3_evaluation' 'HOUSEHOLD_1_459_CA_3_evaluation'
 'HOUSEHOLD_1_334_CA_3_evaluation' 'HOUSEHOLD_1_303_CA_3_evaluation'
 'HOUSEHOLD_1_521_CA_3_evaluation' 'HOUSEHOLD_1_459_WI_2

In [43]:
original_series, prev_ca = pred_manager.get_prediction('CA')
prev_ca.head(10)

,ds,AutoARIMA,ETS,Theta,unique_id,LGBMRegressor,XGBRegressor,LinearRegression
0,2016-06-01,427363.00000,446587.53125,446331.59375,CA,452126.03125,445329.46875,506778.250
1,2016-07-01,441396.71875,446579.65625,447818.28125,CA,440662.93750,457238.25000,476376.000
2,2016-08-01,451900.43750,446573.37500,449304.96875,CA,458109.53125,469236.81250,471009.500
3,2016-09-01,437394.25000,446568.34375,450791.65625,CA,453497.90625,493523.06250,523182.750
4,2016-10-01,445677.06250,446564.31250,452278.34375,CA,447965.28125,461090.93750,474613.500
5,2016-11-01,416368.93750,446561.09375,453765.03125,CA,466381.37500,335757.96875,469907.875


In [41]:
pred_manager.close_connection()

Conexão fechada
